In [1]:
import numpy as np
import os

In [2]:
import os
os.add_dll_directory(r"C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.8\bin")
import ibatensor

In [3]:
def print_tensor_info(label, tensor):
    print(f"{label} shape:", tensor.shape)
    s = tensor.to_string()
    print(s)

In [4]:
data = np.arange(25, dtype=np.float32).reshape(1, 1, 5, 5)
input_tensor = ibatensor.Tensor(data, 1)
print_tensor_info("Input Tensor", input_tensor)

print("\n Testing softmax")
softmax_out = input_tensor.softmax()
print_tensor_info("Softmax Output", softmax_out)

# Create a 1x1x3x3 kernel (out_channels=1, in_channels=1, 3x3)
kernel_data = np.ones((1, 1, 3, 3), dtype=np.float32)
kernel_tensor = ibatensor.Tensor(kernel_data, 1)
print_tensor_info("Kernel", kernel_tensor)

print("\n🧪 Testing conv2d")
conv_output = input_tensor.conv2d(kernel_tensor, padding=1, stride=1)
print_tensor_info("Conv2D Output", conv_output)

print("\n🧪 Testing ReLU")
relu_out = conv_output.relu()
print_tensor_info("ReLU Output", relu_out)

print("\n🧪 Testing avg_pool")
avg_pooled = conv_output.avg_pool(K=2, padding=0, stride=2)
print_tensor_info("Avg Pool Output", avg_pooled)

print("\n🧪 Testing max_pool")
max_pool_result = conv_output.max_pool(K=2, padding=0, stride=2)
print_tensor_info("max_pool", max_pool_result.output)

print("\n🔁 Testing backprop functions")

# Use conv_output as sigma
sigma = conv_output

print("🔧 conv2d_backward_wr_kernel")
grad_kernel = ibatensor.conv2d_backward_wr_kernel(input_tensor, sigma, kernel_tensor, padding=1, stride=1)
print_tensor_info("Grad Kernel", grad_kernel)

print("🔧 conv2d_backward_wr_input")
grad_input = ibatensor.conv2d_backward_wr_input(input_tensor, sigma, kernel_tensor, padding=1, stride=1)
print_tensor_info("Grad Input", grad_input)

print("🔧 max_pool_backward_wr_input")
grad_mp = ibatensor.max_pool_backward_wr_input(conv_output, sigma, max_pool_result.max_inds_ptr, K=2, padding=0, stride=2)
print_tensor_info("Grad Max Pool", grad_mp)

print("🔧 avg_pool_backward_wr_input")
grad_avg = ibatensor.avg_pool_backward_wr_input(conv_output, sigma, K=2, padding=0, stride=2)
print_tensor_info("Grad Avg Pool", grad_avg)

print("🔧 conv2d_backwards_bias_wr_sigma")
grad_bias = ibatensor.conv2d_backwards_bias_wr_sigma(sigma)
print_tensor_info("Grad Bias", grad_bias)

print("🔧 relu_backwards")
grad_relu = ibatensor.relu_backwards(sigma, conv_output)
print_tensor_info("Grad ReLU", grad_relu)

print("🔧 bias_backwards")
grad_b = ibatensor.bias_backwards(sigma)
print_tensor_info("Bias Backward", grad_b)

print("\n✅ All tests complete!")

Input Tensor shape: [1, 1, 5, 5]

=== N=0, C=0 ===
0	1	2	3	4	
5	6	7	8	9	
10	11	12	13	14	
15	16	17	18	19	
20	21	22	23	24	



 Testing softmax
Softmax Output shape: [1, 1, 5, 5]

=== N=0, C=0 ===
0.0116562	0.0316849	0.0861285	0.234122	0.636409	
0.0116562	0.0316849	0.0861285	0.234122	0.636409	
0.0116562	0.0316849	0.0861285	0.234122	0.636409	
0.0116562	0.0316849	0.0861285	0.234122	0.636409	
0.0116562	0.0316849	0.0861285	0.234122	0.636409	


Kernel shape: [1, 1, 3, 3]

=== N=0, C=0 ===
1	1	1	
1	1	1	
1	1	1	



🧪 Testing conv2d
Conv2D Output shape: [1, 1, 5, 5]

=== N=0, C=0 ===
12	21	27	33	24	
33	54	63	72	51	
63	99	108	117	81	
93	144	153	162	111	
72	111	117	123	84	



🧪 Testing ReLU
ReLU Output shape: [1, 1, 5, 5]

=== N=0, C=0 ===
12	21	27	33	24	
33	54	63	72	51	
63	99	108	117	81	
93	144	153	162	111	
72	111	117	123	84	



🧪 Testing avg_pool
Avg Pool Output shape: [1, 1, 2, 2]

=== N=0, C=0 ===
30	48.75	
99.75	135	



🧪 Testing max_pool
max_pool shape: [1, 1, 2, 2]

=== N=0, C=0 ===
54	72	
14

In [9]:
X = (input_tensor @ input_tensor.mat_transpose())

In [18]:
shape = (1, 1,1, 1)  # or whatever shape you want
array = np.full(shape, 0.1, dtype=np.float32)

# Initialize the tensor on GPU (1) or CPU (0)
tensor = ibatensor.Tensor(array, 1)

In [8]:
print((X.elem_wise_mult(learning_rate)).to_string())


=== N=0, C=0 ===
nan	nan	nan	nan	nan	
nan	nan	nan	nan	nan	
nan	nan	nan	nan	nan	
nan	nan	nan	nan	nan	
nan	nan	nan	nan	nan	




In [10]:
print(X.to_string())


=== N=0, C=0 ===
30	80	130	180	230	
80	255	430	605	780	
130	430	730	1030	1330	
180	605	1030	1455	1880	
230	780	1330	1880	2430	




In [19]:
print(tensor.to_string())


=== N=0, C=0 ===
0.1	




In [20]:
print((X.elem_wise_mult(tensor)).to_string())


=== N=0, C=0 ===
3	8	13	18	23	
8	25.5	43	60.5	78	
13	43	73	103	133	
18	60.5	103	145.5	188	
23	78	133	188	243	




In [23]:
print(X.elem_wise_sub(X).to_string())


=== N=0, C=0 ===
0	0	0	0	0	
0	0	0	0	0	
0	0	0	0	0	
0	0	0	0	0	
0	0	0	0	0	




In [61]:
CUDA = 1
class Linear:
    def __init__(self, indim : int, outdim : int):
        weights_initializer = np.random.rand(outdim, indim)
        bias_initializer = np.random.rand(outdim)

        weights_initializer.resize((1,1,outdim, indim))
        self.weights = ibatensor.Tensor(weights_initializer, CUDA)

        bias_initializer.resize((1,1,1,outdim))
        self.bias = ibatensor.Tensor(bias_initializer, CUDA)

        self.prev_update_weights = None
        self.prev_update_bias = None
        self.prev_input = None


    def forward(self, X):
        self.prev_input = X

        return (self.weights @ X).elem_wise_add(self.bias)

    def backward(self, sigma):
        grad_w = sigma @ self.prev_input.mat_transpose()
        grad_b = ibatensor.bias_backwards(sigma)

        if(self.prev_update_weights == None):
            update_weights = grad_w.elem_wise_mult(learning_rate).elem_wise_mult(tensor_neg_one)
            self.weights = self.weights.elem_wise_sub(update_weights)
            update_bias = grad_b.elem_wise_mult(learning_rate).elem_wise_mult(learning_rate)
            self.bias = self.bias.elem_wise_sub(update_bias)

            self.prev_update_weights = update_weights
            self.prev_update_bias = update_bias
        else:
            update_weights = self.prev_update_weights.elem_wise_mult(mew).elem_wise_sub(grad_w.elem_wise_mult(learning_rate).elem_wise_mult(tensor_neg_one))
            self.weights = self.weights.elem_wise_sub(update_weights)
            update_bias = self.prev_update_weights.elem_wise_mult(mew).elem_wise_sub(grad_b.elem_wise_mult(learning_rate).elem_wise_mult(learning_rate))
            self.bias = self.bias.elem_wise_sub(update_bias)

            self.prev_update_weights = update_weights
            self.prev_update_bias = update_bias

        return self.weights.mat_transpose() @ sigma

In [62]:
linear = Linear(3, 5)

In [63]:
input_np = ibatensor.Tensor(np.random.rand(5, 2).astype(np.float32).reshape((1, 1, 5, 2)), CUDA)

In [64]:
out = linear.forward(input_np)

In [65]:
print(out.to_string())


=== N=0, C=0 ===
0.902448	1.45945	
0.63938	1.03205	
1.38612	1.42005	
0.780132	1.27718	
1.39401	1.17011	




In [66]:
sigma_np = np.random.rand(3, 2).astype(np.float32).reshape((1, 1, 3, 2))
sigma_tensor = ibatensor.Tensor(sigma_np, CUDA)

In [67]:
learning_rate = ibatensor.Tensor(np.full((1,1,1,1), 0.1, dtype=np.float32), 1)
mew = ibatensor.Tensor(np.full((1,1,1,1), 0.9, dtype=np.float32), 1)
tensor_neg_one = ibatensor.Tensor(np.full((1,1,1,1), -1, dtype=np.float32), 1)

In [68]:
grad_input = linear.backward(sigma_tensor)

In [69]:
print(grad_input.to_string())


=== N=0, C=0 ===
1.57468	1.42877	
1.26321	1.36643	
1.42836	1.4988	




In [72]:
print(linear.weights.to_string())


=== N=0, C=0 ===
0.647286	0.425714	0.917799	
0.444568	0.0711111	0.297371	
0.761321	0.624283	0.732794	
0.174874	0.546276	0.693221	
0.761664	0.560076	0.15588	




In [73]:
class ReLU():
    def __init__(self):
        self.prev_output = None
        return

    def forward(self, X):
        out = X.relu()
        self.prev_output = out
        return out
        

    def backward(self, sigma):
        return ibatensor.relu_backwards(sigma, self.prev_output) 



In [74]:
linear_relu = ReLU()

In [79]:
new_out = out.elem_wise_add(tensor_neg_one)

In [80]:
print(new_out.to_string())


=== N=0, C=0 ===
-0.0975517	0.459453	
-0.36062	0.0320485	
0.386116	0.420053	
-0.219868	0.277184	
0.394013	0.170113	




In [81]:
activated_out = linear_relu.forward(new_out)

In [82]:
print(activated_out.to_string())


=== N=0, C=0 ===
0	0.459453	
0	0.0320485	
0.386116	0.420053	
0	0.277184	
0.394013	0.170113	




In [83]:
relu_grad = linear_relu.backward(activated_out)

In [84]:
print(relu_grad.to_string())


=== N=0, C=0 ===
0	0.459453	
0	0.0320485	
0.386116	0.420053	
0	0.277184	
0.394013	0.170113	




In [87]:
print(relu_grad.softmax().to_string())


=== N=0, C=0 ===
0.387116	0.612884	
0.491989	0.508011	
0.491517	0.508483	
0.431144	0.568856	
0.555742	0.444258	


